# SDSS Data

Data Release 16 Quasar Catalog v4 fits file

SDSS Schema Browser: http://skyserver.sdss.org/dr16/en/help/browser/browser.aspx.

In [1]:
#Packages to import 
#general
%matplotlib inline
import numpy as np
from astropy.io import fits
import pandas as pd
import collections
import wget
import os
# for SDSS SQL queries
import mechanize
from io import BytesIO


ModuleNotFoundError: No module named 'wget'

In [9]:
#fits_filename = '../data/SDSS/qsoVarPTF-v1_1.fits'
fits_filename = '../data/SDSS/DR16Q_v4.fits'
qsodata = fits.open(fits_filename)

In [10]:
qsodata.info()

Filename: ../data/SDSS/DR16Q_v4.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  CATALOG       1 BinTableHDU    375   750414R x 183C   [18A, D, D, J, J, I, 6A, 6A, I, D, I, D, I, I, I, K, D, I, I, D, I, D, I, D, D, I, D, 12A, D, J, 19A, D, K, D, D, K, D, D, K, D, D, K, D, D, K, D, D, K, D, D, K, D, E, 5D, 5D, 5D, E, D, D, D, D, D, D, D, D, K, K, K, K, K, K, J, J, J, 74J, 74J, 74I, 74I, B, J, 3A, J, J, J, D, D, D, D, 14A, J, D, 5E, 5D, 5E, 5D, 5E, D, D, I, D, D, D, D, I, D, D, D, D, D, D, D, D, E, E, E, E, E, E, E, E, E, I, E, E, E, E, E, E, E, E, E, I, I, D, D, D, D, D, D, J, D, D, D, J, D, D, D, J, D, 21A, D, D, E, E, D, K, D, D, E, E, E, E, E, E, E, D, I, 28A, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D]   


In [43]:
qsodata[1].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 2290 / length of dimension 1                          
NAXIS2  =               750414 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                  183 / number of table fields                         
TTYPE1  = 'SDSS_NAME'                                                           
TFORM1  = '18A     '                                                            
TTYPE2  = 'RA      '                                                            
TFORM2  = 'D       '                                                            
TTYPE3  = 'DEC     '        

In [29]:
qsodata[1].header['TTYPE2']

'RA'

In [68]:
qsodata[1].columns.info('name')

name:
    ['SDSS_NAME', 'RA', 'DEC', 'PLATE', 'MJD', 'FIBERID', 'AUTOCLASS_PQN', 'AUTOCLASS_DR14Q', 'IS_QSO_QN', 'Z_QN', 'RANDOM_SELECT', 'Z_10K', 'Z_CONF_10K', 'PIPE_CORR_10K', 'IS_QSO_10K', 'THING_ID', 'Z_VI', 'Z_CONF', 'CLASS_PERSON', 'Z_DR12Q', 'IS_QSO_DR12Q', 'Z_DR7Q_SCH', 'IS_QSO_DR7Q', 'Z_DR6Q_HW', 'Z_DR7Q_HW', 'IS_QSO_FINAL', 'Z', 'SOURCE_Z', 'Z_PIPE', 'ZWARNING', 'OBJID', 'Z_PCA', 'ZWARN_PCA', 'DELTACHI2_PCA', 'Z_HALPHA', 'ZWARN_HALPHA', 'DELTACHI2_HALPHA', 'Z_HBETA', 'ZWARN_HBETA', 'DELTACHI2_HBETA', 'Z_MGII', 'ZWARN_MGII', 'DELTACHI2_MGII', 'Z_CIII', 'ZWARN_CIII', 'DELTACHI2_CIII', 'Z_CIV', 'ZWARN_CIV', 'DELTACHI2_CIV', 'Z_LYA', 'ZWARN_LYA', 'DELTACHI2_LYA', 'Z_LYAWG', 'Z_DLA', 'NHI_DLA', 'CONF_DLA', 'BAL_PROB', 'BI_CIV', 'ERR_BI_CIV', 'AI_CIV', 'ERR_AI_CIV', 'BI_SIIV', 'ERR_BI_SIIV', 'AI_SIIV', 'ERR_AI_SIIV', 'BOSS_TARGET1', 'EBOSS_TARGET0', 'EBOSS_TARGET1', 'EBOSS_TARGET2', 'ANCILLARY_TARGET1', 'ANCILLARY_TARGET2', 'NSPEC_SDSS', 'NSPEC_BOSS', 'NSPEC', 'PLATE_DUPLICATE', 'M

In [72]:
dataIsQSO = qsodata[1].data.field('IS_QSO_FINAL')
maskIsQSO1 = dataIsQSO == 1
dataIsQSO.shape

(750414,)

In [79]:
dataRA = qsodata[1].data.field('RA')[maskIsQSO1]
dataDEC = qsodata[1].data.field('DEC')[maskIsQSO1]
dataZ = qsodata[1].data.field('Z')[maskIsQSO1]
dataMag = qsodata[1].data.field('PSFMAG')[maskIsQSO1]
dataRA.shape

(749749,)

In [84]:
dataMag.mean(axis=1).shape

(749749,)

In [94]:
d = {'RA': dataRA,
     'DEC': dataDEC,
     'Z': dataZ,
     'MEDIAN_MAG': np.median(dataMag, axis=1)}
df = pd.DataFrame(data=d)
df

,RA,DEC,Z,MEDIAN_MAG
0,0.000629,35.517841,0.845435,18.429541
1,0.001415,31.057048,2.035491,21.562086
2,0.001535,7.064129,1.574227,21.888760
3,0.001526,27.732283,1.770552,21.903051
4,0.001914,9.385637,2.024146,18.499794
...,...,...,...,...
749744,359.999118,28.954734,2.452000,21.349010
749745,359.999303,34.720842,3.109000,19.778521
749746,359.999615,3.268586,1.232962,18.704176
749747,359.999759,20.721079,2.009865,20.267962


In [95]:
df.to_csv('../data/SDSS/RDZM.csv', index=False)